In [1]:
import os
import pandas as pd
from tqdm import trange

In [2]:
year = 2016

In [3]:
data_source = os.path.expanduser('~/data/entso-e/raw')

## Capacity

In [4]:
capacity_df = pd.read_csv('%s/%d_01_InstalledGenerationCapacityAggregated_14.1.A.csv' % (data_source, year + 1), sep='\t')

In [5]:
countries = set(capacity_df[capacity_df['AreaTypeCode'] == 'CTY']['MapCode'].unique())
countries

{'AT',
 'BA',
 'BE',
 'BG',
 'CH',
 'CY',
 'CZ',
 'DE',
 'DK',
 'EE',
 'ES',
 'FI',
 'FR',
 'GB',
 'GR',
 'HR',
 'HU',
 'IE',
 'IT',
 'LT',
 'LU',
 'LV',
 'ME',
 'MK',
 'NL',
 'NO',
 'PL',
 'PT',
 'RO',
 'RS',
 'SE',
 'SI',
 'SK'}

In [6]:
capacity = {country: capacity_df[(capacity_df['MapCode'] == country) & (capacity_df['AreaTypeCode'] == 'CTY')] \
            [['ProductionType', 'AggregatedInstalledCapacity']].set_index('ProductionType').T.to_dict('records')[0]
            for country in countries}

In [7]:
capacity['FR']

{'Fossil Gas': 6696.0,
 'Biomass': 170.0,
 'Other': 62.0,
 'Solar': 7660.0,
 'Hydro Water Reservoir': 8230.66,
 'Fossil Oil': 5300.0,
 'Marine': 240.0,
 'Hydro Run-of-river and poundage': 10326.98,
 'Nuclear': 63130.0,
 'Fossil Hard coal': 2930.0,
 'Wind Offshore': 0.0,
 'Wind Onshore': 13569.0,
 'Hydro Pumped Storage': 4965.0}

In [8]:
capacity['DE']

{'Fossil Brown coal/Lignite': 21262.0,
 'Biomass': 7080.25,
 'Fossil Hard coal': 27437.0,
 'Waste': 1685.0,
 'Geothermal': 40.22,
 'Other': 1421.0,
 'Hydro Water Reservoir': 1439.0,
 'Fossil Oil': 4614.0,
 'Wind Offshore': 4131.3,
 'Fossil Coal-derived gas': 0.0,
 'Fossil Gas': 32627.0,
 'Hydro Run-of-river and poundage': 4007.13,
 'Other renewable': 509.61,
 'Hydro Pumped Storage': 8894.24,
 'Nuclear': 10793.0,
 'Wind Onshore': 47042.0,
 'Solar': 40834.46}

In [9]:
capacity['CH']

{'Hydro Run-of-river and poundage': 190.0,
 'Nuclear': 3373.0,
 'Hydro Water Reservoir': 4897.8,
 'Hydro Pumped Storage': 6228.0}

In [10]:
all_capacity_types = set([])
for cap in capacity.values():
    all_capacity_types.update(cap.keys())
all_capacity_types

{'Biomass',
 'Fossil Brown coal/Lignite',
 'Fossil Coal-derived gas',
 'Fossil Gas',
 'Fossil Hard coal',
 'Fossil Oil',
 'Fossil Oil shale',
 'Fossil Peat',
 'Geothermal',
 'Hydro Pumped Storage',
 'Hydro Run-of-river and poundage',
 'Hydro Water Reservoir',
 'Marine',
 'Nuclear',
 'Other',
 'Other renewable',
 'Solar',
 'Waste',
 'Wind Offshore',
 'Wind Onshore'}

In [11]:
total_capacity = {type: sum([capacity[country][type] for country in countries if type in capacity[country].keys()])
                  for type in all_capacity_types}
total_capacity

{'Geothermal': 927.12,
 'Other renewable': 3852.59,
 'Fossil Coal-derived gas': 1569.0,
 'Fossil Oil shale': 1976.0,
 'Fossil Gas': 206834.42000000004,
 'Solar': 74883.42,
 'Waste': 4376.84,
 'Marine': 240.0,
 'Fossil Peat': 1420.6,
 'Fossil Hard coal': 97463.93,
 'Hydro Water Reservoir': 98348.26000000001,
 'Hydro Pumped Storage': 51659.56,
 'Wind Offshore': 12707.5,
 'Fossil Brown coal/Lignite': 59820.03,
 'Wind Onshore': 143804.99,
 'Nuclear': 124003.19,
 'Fossil Oil': 24607.300000000003,
 'Hydro Run-of-river and poundage': 54860.310000000005,
 'Other': 19761.84,
 'Biomass': 17475.870000000003}

## Generation

In [12]:
gen_sum = {country : {t: 0.0 for t in type.keys()} for (country, type) in capacity.items()}
gen_count = {country : {t: 0 for t in type.keys()} for (country, type) in capacity.items()}
for month in trange(1, 13):
    gen_df = pd.read_csv('%s/%d_%02d_AggregatedGenerationPerType_16.1.B_C.csv' % (data_source, year, month), sep='\t')
    for (country, cap) in capacity.items():
        country_gen_df = gen_df[(gen_df['MapCode'] == country) & (gen_df['AreaTypeCode'] == 'CTY')]
        for type in cap.keys():
            timestep_gen = country_gen_df[country_gen_df['ProductionType'] == type]['ActualGenerationOutput']
            gen_sum[country][type] += timestep_gen.sum()
            gen_count[country][type] += len(timestep_gen)

100%|███████████████████████████████████████████| 12/12 [00:53<00:00,  4.45s/it]


In [13]:
gen = {country: {type: gen_sum[country][type] / count for (type, count) in g.items() if count > 0}
       for (country, g) in gen_count.items()}

In [14]:
countries - set(gen.keys())

set()

In [15]:
all_types = set([])
for g in gen.values():
    all_types.update(g.keys())
all_types == all_capacity_types

True

In [16]:
total_gen = {type: sum([gen[country][type] for country in countries if type in gen[country].keys()]) for type in all_types}
total_gen

{'Geothermal': 685.6604957877961,
 'Other renewable': 590.3135386805076,
 'Fossil Coal-derived gas': 992.0888080453301,
 'Fossil Oil shale': 965.3841072447235,
 'Fossil Gas': 49673.17116282833,
 'Solar': 9948.528703189875,
 'Waste': 1654.7528716305037,
 'Marine': 0.0,
 'Fossil Peat': 731.3962567361177,
 'Fossil Hard coal': 35701.424692876215,
 'Hydro Water Reservoir': 31623.565299100464,
 'Hydro Pumped Storage': 4519.202029395663,
 'Wind Offshore': 3673.6678082143408,
 'Fossil Brown coal/Lignite': 34858.662974484774,
 'Wind Onshore': 28541.914258691355,
 'Nuclear': 93201.6588195912,
 'Fossil Oil': 1975.104576008247,
 'Hydro Run-of-river and poundage': 22670.008752384645,
 'Other': 18080.785224791332,
 'Biomass': 8043.577574139039}

In [17]:
for country in countries:
    for type in gen[country].keys():
        if type not in capacity[country].keys():
            print('Missing capacity for type "%s" in country %s' % (type, country))
        elif gen[country][type] > capacity[country][type]:
            print('Production (%.2f) above capacity (%.2f) for type "%s" in country %s'
                  % (gen[country][type], capacity[country][type], type, country))

Production (3912.27) above capacity (1.00) for type "Other" in country NL
Production (6.93) above capacity (0.00) for type "Fossil Oil" in country CZ
Production (109.33) above capacity (0.00) for type "Other" in country CZ
Production (6.70) above capacity (6.00) for type "Other renewable" in country EE
Production (4.55) above capacity (0.00) for type "Fossil Peat" in country EE
Production (10472.35) above capacity (1708.00) for type "Other" in country IT
Production (0.27) above capacity (0.00) for type "Fossil Oil" in country GR
Production (11.55) above capacity (1.00) for type "Solar" in country LU
Production (7.21) above capacity (2.00) for type "Biomass" in country LU
Production (284.76) above capacity (170.00) for type "Biomass" in country FR
Production (380.71) above capacity (0.00) for type "Fossil Brown coal/Lignite" in country ES
Production (30.05) above capacity (0.00) for type "Other" in country NO
Production (431.82) above capacity (0.00) for type "Fossil Coal-derived gas" i

## Usage

In [18]:
usage = {country: {type: min(1.0, gen[country][type] / c) for (type, c) in cap.items() if (c > 0) and (type in gen[country].keys())}
         for (country, cap) in capacity.items()}

In [19]:
usage['CH']

{'Hydro Run-of-river and poundage': 0.36151283757145675,
 'Nuclear': 0.6655099061604068,
 'Hydro Water Reservoir': 0.1847211723009944,
 'Hydro Pumped Storage': 0.09379069662187217}

In [20]:
usage['FR']

{'Fossil Gas': 0.6103885420147693,
 'Biomass': 1.0,
 'Solar': 0.11872979183804856,
 'Hydro Water Reservoir': 0.2077282539569099,
 'Fossil Oil': 0.050642117381256256,
 'Hydro Run-of-river and poundage': 0.4655437275228686,
 'Nuclear': 0.6902363886408641,
 'Fossil Hard coal': 0.2791861530960507,
 'Wind Onshore': 0.16459696226751308,
 'Hydro Pumped Storage': 0.11771793481119061}

In [21]:
usage['DE']

{'Fossil Brown coal/Lignite': 0.6980634995721477,
 'Biomass': 0.6390688081340326,
 'Fossil Hard coal': 0.33563320772100996,
 'Waste': 0.3711244066295341,
 'Geothermal': 0.477981876376197,
 'Other': 0.21764943061500083,
 'Hydro Water Reservoir': 0.06481778794061095,
 'Fossil Oil': 0.04291615512412528,
 'Wind Offshore': 0.33326902711967815,
 'Fossil Gas': 0.07986534508272647,
 'Hydro Run-of-river and poundage': 0.4902638078306201,
 'Other renewable': 0.2757552750524714,
 'Hydro Pumped Storage': 0.10991873654739151,
 'Nuclear': 0.8463433163183248,
 'Wind Onshore': 0.15798061646690328,
 'Solar': 0.0962918313103897}

## Aggregating for PanTaGruEl

In [22]:
PanTaGruEl_countries = {"BG", "BE", "IT", "HU", "FR", "AT", "DE", "PL", "CZ", "NL",
                        "GR", "CH", "RO", "ES", "SI", "PT", "LU", "HR", "AL", "DK",
                        "RS", "SK", "BA", "ME", "MK"}

In [23]:
set(PanTaGruEl_countries) - set(countries)

{'AL'}

In [24]:
types_aggregation = {
    'Fossil Brown coal/Lignite'       : 'coal',
    'Fossil Hard coal'                : 'coal',
    'Fossil Coal-derived gas'         : 'gas',
    'Fossil Gas'                      : 'gas',
    'Fossil Oil'                      : 'gas',
    'Fossil Oil shale'                : 'gas',
    'Fossil Peat'                     : 'coal',
    'Hydro Pumped Storage'            : 'hydro_storage',
    'Hydro Water Reservoir'           : 'hydro_storage',
    'Hydro Run-of-river and poundage' : 'hydro_ror',
    'Nuclear'                         : 'nuclear',
    'Biomass'                         : 'other',
    'Geothermal'                      : 'other',
    'Marine'                          : 'other',
    'Other'                           : 'other',
    'Other renewable'                 : 'other',
    'Solar'                           : 'other',
    'Waste'                           : 'other',
    'Wind Offshore'                   : 'other',
    'Wind Onshore'                    : 'other'
}

all_types == set(types_aggregation.keys())

True

In [25]:
aggregated_types = set(types_aggregation.values())
aggregated_types

{'coal', 'gas', 'hydro_ror', 'hydro_storage', 'nuclear', 'other'}

In [26]:
len(PanTaGruEl_countries)

25

In [27]:
valid_countries = PanTaGruEl_countries.intersection(countries)
aggregated_gen = {country: {type: 0.0 for type in aggregated_types} for country in valid_countries}
aggregated_capacity = {country: {type: 0.0 for type in aggregated_types} for country in valid_countries}

for country in valid_countries:
    for (type, value) in gen[country].items():
        aggregated_gen[country][types_aggregation[type]] += value
    for (type, value) in capacity[country].items():
        aggregated_capacity[country][types_aggregation[type]] += value

    aggregated_gen[country]["hydro"] = aggregated_gen[country]["hydro_storage"] + aggregated_gen[country]["hydro_ror"]
    aggregated_capacity[country]["hydro"] = aggregated_capacity[country]["hydro_storage"] + aggregated_capacity[country]["hydro_ror"]

In [28]:
aggregated_usage = {country: {type: min(1.0, aggregated_gen[country][type] / c) if c > 0 else 0.0 for (type, c) in cap.items()}
                    for (country, cap) in aggregated_capacity.items()}

Compute average usage by type:

In [29]:
def list_avg(x):
    return sum(x) / len(x)
average_aggregated_usage = {type: list_avg([usage[type] for country, usage in aggregated_usage.items() if usage[type] > 0.0])
                            for type in aggregated_types.union({'hydro'})}
average_aggregated_usage

{'gas': 0.26623399035193107,
 'hydro': 0.24781086015765597,
 'hydro_storage': 0.172184816632542,
 'nuclear': 0.8261343974102862,
 'hydro_ror': 0.40586266156694306,
 'coal': 0.4665722228670449,
 'other': 0.2911420353616552}

- Albania is not part of the ENTSO-E data
- Bosnia and Croatia have no usage data

For these countries, use average value

In [30]:
for country in {'AL', 'BA', 'HR'}:
    aggregated_usage[country] = average_aggregated_usage

In [31]:
aggregated_usage_df = pd.DataFrame.from_dict(aggregated_usage)
aggregated_usage_df.T

,nuclear,hydro_ror,gas,coal,hydro_storage,other,hydro
NL,0.878398,0.000000,0.202746,0.779267,0.000000,0.568052,0.000000
CZ,0.641604,0.280609,0.385261,0.476716,0.143951,0.290758,0.164156
PL,0.000000,0.451889,0.438759,0.544513,0.047969,0.228897,0.116831
HU,0.913563,0.402413,0.137389,0.597108,0.494236,0.253345,0.448341
DK,0.000000,0.000000,0.128388,0.224035,0.000000,0.270805,0.000000
IT,0.000000,0.339572,0.160118,0.144434,0.093580,0.851337,0.210163
PT,0.000000,0.380341,0.284405,0.761095,0.200312,0.305767,0.274320
BG,0.898148,0.100629,0.598930,0.562805,0.193433,0.185281,0.178081
BA,0.826134,0.405863,0.266234,0.466572,0.172185,0.291142,0.247811
SI,0.888306,0.471659,0.063636,0.554624,0.175480,0.184435,0.428422


In [32]:
aggregated_usage_df.to_csv('pantagruel_capacity_usage.csv')